# Apartment Visualization Helper

**Goal**: Visualize apartment layouts and furniture placement

This notebook will help you:
- Plot room boundaries
- Visualize furniture positions
- Compare empty vs furnished apartments
- Debug geometric placement

In [ ]:
import sys
sys.path.append('../utils')

import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon, Rectangle, FancyBboxPatch
import numpy as np
from pathlib import Path

from json_processing import load_apartment_json, extract_furniture_ids
from furniture_catalog import get_furniture_dims

plt.rcParams['figure.figsize'] = (12, 8)

## Helper Functions for Plotting

In [ ]:
def plot_boundary(ax, boundary, color='black', linewidth=2, label=None):
    """Plot room boundary (contour + islands)."""
    if not boundary:
        return
    
    # Plot contour
    contour = boundary.get('contour', [])
    if contour:
        contour_array = np.array(contour)
        ax.plot(contour_array[:, 0], contour_array[:, 1], 
               color=color, linewidth=linewidth, label=label)
        # Close the polygon
        ax.plot([contour_array[-1, 0], contour_array[0, 0]], 
               [contour_array[-1, 1], contour_array[0, 1]], 
               color=color, linewidth=linewidth)
    
    # Plot islands (holes)
    islands = boundary.get('islands', [])
    for island in islands:
        island_array = np.array(island)
        ax.plot(island_array[:, 0], island_array[:, 1], 
               color=color, linewidth=linewidth, linestyle='--')


def plot_furniture(ax, furniture_item, catalog=None, color='blue', alpha=0.5):
    """Plot single furniture item as rectangle."""
    furn_id = furniture_item.get('id', 'unknown')
    insert_point = furniture_item.get('insert_point', [0, 0])
    angle = furniture_item.get('angle', 0)
    x_scale = furniture_item.get('x_scale', 1.0)
    y_scale = furniture_item.get('y_scale', 1.0)
    
    # Get dimensions from catalog or use defaults
    if catalog and furn_id in catalog:
        dims = catalog[furn_id]
        width = dims['width'] * x_scale * 100  # Convert to cm for plotting
        height = dims['height'] * y_scale * 100
    else:
        # Default size if not in catalog
        width = 100 * x_scale
        height = 100 * y_scale
    
    # Create rectangle at insert point
    rect = FancyBboxPatch(
        insert_point, width, height,
        boxstyle="round,pad=0.05",
        linewidth=1.5,
        edgecolor=color,
        facecolor=color,
        alpha=alpha,
        angle=angle
    )
    ax.add_patch(rect)
    
    # Add label
    ax.text(insert_point[0] + width/2, insert_point[1] + height/2, 
           furn_id, ha='center', va='center', fontsize=8, color='white', weight='bold')


def plot_compartment(ax, compartment, catalog=None, show_furniture=True):
    """Plot single room with furniture."""
    room_name = compartment.get('name_cad', 'unknown')
    boundary = compartment.get('boundary', {})
    
    # Plot boundary
    plot_boundary(ax, boundary, color='black', linewidth=2, label=room_name)
    
    # Plot furniture if requested
    if show_furniture:
        for equipment in compartment.get('equipments', []):
            plot_furniture(ax, equipment, catalog, color='steelblue')
        
        for casework in compartment.get('caseworks', []):
            plot_furniture(ax, casework, catalog, color='coral')

## Load Sample Apartment

In [ ]:
# TODO: Update to your sample file
apartment_file = "../samples/apartment_001.json"

if Path(apartment_file).exists():
    apt_data = load_apartment_json(apartment_file)
    print("✅ Loaded apartment")
    
    # Show structure
    for apt in apt_data.get('apartments', []):
        print(f"\nApartment: {apt.get('id_cad', 'unknown')}")
        print(f"Type: {apt.get('type', 'unknown')}")
        print(f"Rooms: {len(apt.get('compartments', []))}")
else:
    print(f"⚠️  File not found: {apartment_file}")
    print("Upload sample apartments tomorrow!")

## Load Furniture Catalog

In [ ]:
# Load catalog if available
catalog_file = "../samples/furniture_catalog.json"

catalog = None
if Path(catalog_file).exists():
    with open(catalog_file, 'r') as f:
        catalog = json.load(f)
    print(f"✅ Loaded catalog with {len(catalog)} furniture items")
else:
    print("⚠️  Catalog not found. Run extract_furniture_catalog.ipynb first.")
    print("Using default dimensions for visualization.")

## Visualize All Rooms

In [ ]:
if 'apt_data' in locals():
    for apt in apt_data.get('apartments', []):
        compartments = apt.get('compartments', [])
        
        # Create subplots for each room
        n_rooms = len(compartments)
        if n_rooms == 0:
            continue
        
        fig, axes = plt.subplots(1, min(n_rooms, 4), figsize=(16, 5))
        if n_rooms == 1:
            axes = [axes]
        
        for idx, comp in enumerate(compartments[:4]):  # Show first 4 rooms
            ax = axes[idx] if n_rooms > 1 else axes[0]
            
            plot_compartment(ax, comp, catalog, show_furniture=True)
            
            room_name = comp.get('name_cad', 'unknown')
            area = comp.get('area', 0)
            ax.set_title(f"{room_name}\n{area:.1f}m²", fontsize=10)
            ax.set_aspect('equal')
            ax.grid(True, alpha=0.3)
            ax.set_xlabel('X (cm)')
            ax.set_ylabel('Y (cm)')
        
        plt.tight_layout()
        plt.show()
        
        if n_rooms > 4:
            print(f"\n(Showing first 4 of {n_rooms} rooms)")

## Compare Empty vs Furnished

Side-by-side comparison (useful for debugging Stage 2 geometric placement)

In [ ]:
if 'apt_data' in locals():
    # Pick first room
    for apt in apt_data.get('apartments', []):
        compartments = apt.get('compartments', [])
        if not compartments:
            continue
        
        comp = compartments[0]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        # Empty room
        plot_compartment(ax1, comp, catalog, show_furniture=False)
        ax1.set_title(f"Empty: {comp.get('name_cad', 'unknown')}", fontsize=12)
        ax1.set_aspect('equal')
        ax1.grid(True, alpha=0.3)
        
        # Furnished room
        plot_compartment(ax2, comp, catalog, show_furniture=True)
        ax2.set_title(f"Furnished: {comp.get('name_cad', 'unknown')}", fontsize=12)
        ax2.set_aspect('equal')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        break  # Only first apartment

## Next Steps

This visualization will be useful for:
1. Debugging geometric placement (Stage 2)
2. Validating generated furniture positions
3. Human evaluation of results
4. Creating examples for documentation